In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [158]:
import numpy as np
import pandas as pd
import inspect
import types
import pickle

from types import SimpleNamespace
import operator
from typing import Generator
import itertools

from test import\
    test_eq,\
    test_is,\
    test_fail,\
    is_iter,\
    test_ne,\
    test_shuffled,\
    test_eq_type

from core import\
    FixSigMeta,\
    PrePostInitMeta,\
    NewChkMeta,\
    BypassNewMeta,\
    patch_to,\
    patch,\
    patch_property,\
    use_kwargs,\
    delegates,\
    funcs_kwargs,\
    method,\
    add_docs,\
    GetAttr,\
    delegate_attr,\
    coll_repr,\
    mask2idxs,\
    cycle,\
    zip_cycle,\
    L,\
    _listify,\
    _is_array,\
    ifnone,\
    get_class,\
    mk_class,\
    wrap_class,\
    noop,\
    noops,\
    store_attr,\
    attrdict

## Metaclasses 

In [3]:
class _T(metaclass=PrePostInitMeta):
    def __pre_init__(self):
        self.a = 0
        assert self.a == 0
    def __init__(self):
        self.a += 1
        assert self.a == 1
    def __post_init__(self):
        self.a += 1
        assert self.a == 2
        
t = _T()
test_eq(t.a, 2)

In [4]:
t = _T()
isinstance(t, _T)

True

In [5]:
class _T(metaclass=NewChkMeta):
    "Testing"
    def __init__(self, o=None, b=1):
        self.foo = getattr(o, "foo", 0) + 1
        self.b = b
        
class _T2:
    "Testing 2"
    def __init__(self, o):
        self.foo = getattr(o, "foo", 0) + 1
        
t = _T(1)
test_eq(t.foo, 1)
t2 = _T(t)
test_eq(t2.foo, 1)
test_is(t, t2)
t3 = _T(t, b=2)
test_eq(t3.b, 2)
assert not t3 is t

t = _T2(1)
test_eq(t.foo, 1)
t2 = _T2(t)
test_eq(t2.foo,2)
assert not t2 is t

test_eq(_T.__doc__, "Testing")
test_eq(str(inspect.signature(_T)), "(self, o=None, b=1)")

In [6]:
class T0: pass
class _T(T0, metaclass=BypassNewMeta):
    _bypass_type=T0
    def __init__(self, x): self.x=x
        
t = T0()
t.a = 1
t2 = _T(t)
test_eq(type(t2), _T)
test_eq(t2.a, 1)
test_is(t2, t)
t = _T(2)
t.x = 2

## Foundational functions

In [7]:
class _T3(int): pass

@patch_to(_T3)
def func1(x, a): return x+a

t = _T3(1)
test_eq(t.func1(2), 3)

In [8]:
class _T4(int): pass
@patch_to((_T3, _T4))
def func2(x, a): return x+2*a

t = _T3(1)
test_eq(t.func2(1), 3)
t = _T4(1)
test_eq(t.func2(1), 3)

In [9]:
def foo(val:str)->None:
    return "hi {val}"

foo.__annotations__.values()

dict_values([<class 'str'>, None])

In [10]:
@patch
def func(x:_T3, a):
    "test"
    return x+2

t = _T3(1)
test_eq(t.func(2), 3)
test_eq(t.func.__qualname__, "_T3.func")

In [11]:
@patch
def func3(x:(_T3, _T4), a):
    "test"
    return x+2*a

t = _T3(1)
test_eq(t.func3(2), 5)
test_eq(t.func3.__qualname__, "_T3.func3")
t = _T4(1)
test_eq(t.func3(2), 5)
test_eq(t.func3.__qualname__, "_T4.func3")

In [12]:
inspect.signature(t.func3)

<Signature (a)>

In [13]:
class foo:
    def bar(self, x):
        return x+1
inspect.signature(foo().bar)

<Signature (x)>

In [14]:
@patch_property
def prop(x:_T3): return x+1

t = _T3(1)
test_eq(t.prop, 2)

In [15]:
inspect.Parameter("car", inspect.Parameter.KEYWORD_ONLY)

<Parameter "car">

In [16]:
def test_sig(func, b): test_eq(str(inspect.signature(func)), b)

In [17]:
@use_kwargs(["y", "z"])
def foo(a, b=1, **kwargs): pass
test_sig(foo, '(a, b=1, *, y=None, z=None)')

@use_kwargs(["y", "z"], keep=True)
def foo(a, *args, b=1, **kwargs): pass
test_sig(foo, '(a, *args, b=1, y=None, z=None, **kwargs)')

In [18]:
def basefoo(e, c=2): pass

@delegates(basefoo)
def foo(a, b=1, **kwargs): pass
test_sig(foo, '(a, b=1, c=2)')

@delegates(basefoo, keep=True)
def foo(a, b=1, **kwargs): pass
test_sig(foo, '(a, b=1, c=2, **kwargs)')

In [19]:
@funcs_kwargs
class T:
    _methods=["b"]
    def __init__(self, f=1, **kwargs): assert not kwargs
    def a(self): return 1
    def b(self): return 2
    
t = T()
test_eq(t.a(), 1)
test_eq(t.b(), 2)
t = T(b=lambda:3)
test_eq(t.b(), 3)
test_sig(T, '(f=1, *, b=None)')
test_fail(lambda: T(a = lambda:3))

@method
def _f(self, a=1): return a+1
t = T(b = _f)
test_eq(t.b(2), 3)

class T2(T):
    def __init__(self, a):
        super().__init__(b = lambda: 3)
        self.a = a
t = T2(a=1)
test_eq(t.b(), 3)
test_sig(T2, '(a)')

def _g(a=1): return a+1
class T3(T): b = staticmethod(_g)
t = T3()
test_eq(t.b(2), 3)

In [20]:
def is_listy(x): return isinstance(x, (list, tuple, Generator))

In [21]:
class _T:
    def f(self): pass
    @classmethod
    def g(cls): pass
add_docs(_T, "a", f="f", g="g")

test_eq(_T.__doc__, "a")
test_eq(_T.f.__doc__, "f")
test_eq(_T.g.__doc__, "g")

In [22]:
assert is_iter([1])
assert not is_iter(np.array(1))
assert is_iter(np.array([1,2]))
assert (o for o in range(3))

## GetAttr 

In [23]:
class _C(GetAttr):
    _xtra = ['lower']
    def __init__(self, a): self.default = a
    def foo(self): noop
    def test(self): return "low"
        
t = _C("Hi")
test_eq(t.lower(), "hi")
test_fail(lambda: t.upper())
assert "lower" in dir(t)

In [24]:
class _C:
    f = "Hi"
    def __getattr__(self, k): return delegate_attr(self, k, "f")
    
t = _C()
test_eq(t.lower(), "hi")

## L- 

In [25]:
coll_repr(range(1000), 10)

'(#1000) [0,1,2,3,4,5,6,7,8,9...]'

In [26]:
coll_repr(range(5), 10)

'(#5) [0,1,2,3,4]'

In [27]:
test_eq(mask2idxs([False, True, False, True]), [1,3])
test_eq(mask2idxs(np.array([1, 2, 3])), [1, 2, 3])

In [28]:
test_eq(itertools.islice(cycle([1,2,3]), 5), [1,2,3,1,2])
test_eq(itertools.islice(cycle([]),3), [None]*3)
test_eq(itertools.islice(cycle(None), 3), [None]*3)
test_eq(itertools.islice(cycle(1), 3), [1,1,1])

In [29]:
test_eq(zip_cycle([1,2,3,4], list("abc")), [(1, "a"), (2, "b"), (3, "c"), (4, "a")])

In [30]:
t = L(range(12))
test_eq(t, list(range(12)))
test_ne(t, list(range(11)))
t.reverse() # uses list methods (clever!)
test_eq(t[0], 11)
t[3] = "h"
t[3,5] = ("j", "k") # __setitem__ modified
test_eq(t[3,5], ["j", "k"])
test_eq(t, L(t))
test_eq(L(L(1,2), [3,4]), ([1,2], [3,4])) # arg is tuple ([3,4],)
t

(#12) [11,10,9,j,7,k,5,4,3,2...]

In [31]:
arr = np.arange(9).reshape(3,3)
t = L(arr, use_list=None) # uses numpy as reference
test_eq(t[1,2], arr[[1,2]])

df = pd.DataFrame({"a":[1,2,3]})
t = L(df, use_list=None)
test_eq(t[1,2], L(pd.DataFrame({"a":[2,3]}, index=[1,2]), use_list=None))

In [32]:
t = L()
test_eq(t, [])
t.append(1)
test_eq(t, [1])
t += [3,2]
test_eq(t, [1,3,2])
t += [4]
test_eq(t, [1,3,2,4])
t = 5 + t
test_eq(t, [5,1,3,2,4])
test_eq(L(1,2,3), [1,2,3]) # (items,) + (2,3)
test_eq(L(1,2,3), L(1,2,3))
t = L(1)*5
t = t.map(operator.neg)
test_eq(t, [-1]*5)
test_eq(~L([True, False, False]), L([False, True, True]))
t = L(range(4))
test_eq(zip(t, L(1).cycle()), zip(range(4), (1,1,1,1)))
t = L.range(100)
test_shuffled(t, t.shuffled())

In [33]:
def _f(x, a=0): return x+a
t = L(1)*5
test_eq(t.map(_f), t)
test_eq(t.map(_f, 1), [2]*5)
test_eq(t.map(_f, a=2), [3]*5)

In [34]:
test_eq(L([1,2,3]), [1,2,3])
test_eq(L(L(1,2,3)), [1,2,3])
test_ne(L([1,2,3]), [1,2,])
test_eq(L("abc"), ["abc"])
test_eq(L(range(0,3)), [0,1,2])
test_eq(L(o for o in range(0,3)), [0,1,2])
test_eq(L(np.array(0)), [np.array(0)])
test_eq(L([np.array(0), np.array(1)]), [np.array(0), np.array(1)])
test_eq(L(np.array([0.,1.1]))[0], np.array([0.,1.1]))
test_eq(L(np.array([0,1.1]), use_list=True), [np.array(0.), np.array(1.1)])

In [35]:
test_eq(L(1, match=[1,2,3]), [1,1,1])
test_eq(L([1,2], match=[2,3]), [1,2])
test_fail(lambda: L([1,2], match=[1,2,3]))

In [36]:
test_is(L(t), t)

In [37]:
test_eq(L(["a", "b"]), ["a", "b"])
test_ne(L(["a", "b"]), "ab")
test_ne(L(["a", "b"]), {"a", "b"})
test_ne(L(["a", "b"]), {"a":1, "b":2})

In [38]:
t = L(range(12))
test_eq(t[1,2], [1,2])
test_eq(t[[1,2]], [1,2])
test_eq(t[:3], [0,1,2])
test_eq(t[[False]*11 + [True]], [11])
test_eq(t[np.array((3))], 3)

In [39]:
t[4,6] = 0
test_eq(t[4,6], [0,0])
t[4,6] = [1,2]
test_eq(t[4,6], [1,2])

In [40]:
test_eq(L(1,2,3,4,4).unique(), [1,2,3,4])

In [41]:
test_eq(L(1,2,3).val2idx(), {3:2,1:0,2:1})

In [42]:
test_eq(t.filter(lambda obj: obj < 5), [0,1,2,3,1,2])

In [43]:
test_eq(L.range(4).map(operator.neg), [0,-1,-2,-3])

In [44]:
test_eq(L.range(4).map("#{}#"), ["#0#", "#1#", "#2#", "#3#"])

In [45]:
test_eq(L.range(4).map(list("abcd")), list("abcd"))

In [46]:
test_eq(L(range(1,5)).map_dict(operator.neg), {1:-1, 2:-2, 3:-3, 4:-4})

In [47]:
t = L([[1,2,3], "abc"])
test_eq(t.zip(), [(1,"a"), (2, "b"), (3, "c")])

In [48]:
t = L([[1,2,3,4], ["a","b","c"]])
test_eq(t.zip(cycled=True ), [(1, "a"), (2, "b"), (3, "c"), (4, "a")])
test_eq(t.zip(cycled=False), [(1, "a"), (2, "b"), (3, "c")])

In [49]:
t = L([1, 2, 3], [2, 3, 4])
test_eq(t.map_zip(operator.mul), [2, 6, 12])

In [53]:
b = [[0], [1], [2,2]]
t = L([1,2,3]).zipwith(b)
test_eq(t, [(1, [0]), (2, [1]), (3, [2,2])])

In [56]:
test_eq(L(1,2,3).map_zipwith(operator.mul, [2, 3, 4]), [2, 6, 12])

In [57]:
test_eq(t.itemgot(1), b)

In [68]:
# SimpleNamespace is a light class provides readable repr
a = [SimpleNamespace(a=3, b=4), SimpleNamespace(a=1, b=2)]
test_eq(L(a).attrgot("b"), [4, 2])

In [69]:
test_eq(L.split("a b c"), list("abc"))

In [76]:
test_eq_type(L.range([1,1,1]), L(range(3)))
test_eq_type(L.range(5,2,2), L(range(5,2,2)))

In [80]:
test_eq(L([0, 1, 2, 3], 4, L(5, 6)).concat(), range(7))

In [81]:
L([0, 1, 2, 3], 4, L(5, 6)).concat()

(#7) [0,1,2,3,4,5,6]

## Utility Functions 

In [85]:
test_eq(ifnone(None, 1), 1)
test_eq(ifnone(2,    1), 2)

In [100]:
_t = get_class("_t", "a", b=2)
t = _t()
test_eq(t.a, None)
test_eq(t.b, 2)
t = _t(1, b=3)
test_eq(t.a, 1)
test_eq(t.b, 3)
t = _t(1, 3)
test_eq(t.a, 1)
test_eq(t.b, 3)

In [114]:
mk_class("_t", a=1, sup=GetAttr)
t = _t()
test_eq(t.a, 1)
assert(isinstance(t, GetAttr))

In [133]:
def foo(self): return 1
mk_class("_t", "a", sup=GetAttr, doc="test doc", funcs=foo)

t = _t(3, b=2)
test_eq(t.a, 3)
test_eq(t.b, 2)
test_eq(t.foo(), 1)
test_eq(t.__doc__, "test doc")
t

In [135]:
@wrap_class("_t", a=2)
def bar(self, x): return x+1

t = _t()
test_eq(t.a, 2)
test_eq(t.bar(3), 4)

In [153]:
noop()
test_eq(noop(1), 1)

In [154]:
mk_class("_t", foo=noops)
test_eq(_t().foo(1), 1)

In [157]:
class T:
    def __init__(self, a, b, c): store_attr(self, 'a,b, c')
        
t = T(1,c=2,b=3)
assert t.a == 1 and t.b == 3 and t.c == 2

In [159]:
test_eq(attrdict(t, "b", "c"), {"b":3, "c":2})

In [ ]:
class T:
    def a(self): return 1
    def b(self): return 2
properties(T, "a")